In [7]:
# ---------------------------------------------------------------
# Landsat 8 Summer NDVI — New England (state-by-state)
# - Collection: LANDSAT/LC08/C02/T1_L2 (30 m, SR)
# - NDVI scaled by 10000 and cast to Int16 (saves space)
# - One GeoTIFF per state, median over June–August for YEAR
# ---------------------------------------------------------------
# Setup:
#   pip install earthengine-api
#   earthengine authenticate
# ---------------------------------------------------------------

import ee
ee.Initialize()

# ---------------- Config (edit these) ----------------
YEAR         = 2020                 # any L8 year >= 2013
DRIVE_FOLDER = "DATA_EE_DOWNLOAD"  # Google Drive folder
SCALE_M      = 30                   # meters
CRS          = None                 # e.g., "EPSG:5070" or None for native

# Hardcoded New England states (USPS codes)
NEW_ENGLAND = ["ME", "NH", "VT", "MA", "CT", "RI"]

# ------------- Helpers (minimal, readable) -------------
def mask_l8_qapixel(img):
    # Mask dilated cloud, cirrus, cloud, cloud shadow, snow (QA_PIXEL bits 1..5)
    qa = img.select("QA_PIXEL")
    good = (
        qa.bitwiseAnd(1 << 1).eq(0)  # dilated cloud
        .And(qa.bitwiseAnd(1 << 2).eq(0))  # cirrus
        .And(qa.bitwiseAnd(1 << 3).eq(0))  # cloud
        .And(qa.bitwiseAnd(1 << 4).eq(0))  # cloud shadow
        .And(qa.bitwiseAnd(1 << 5).eq(0))  # snow
    )
    return img.updateMask(good)

def add_ndvi_scaled(img):
    # Convert L2 SR to reflectance (DN * 2.75e-05 - 0.2), then NDVI, then scale*10000 -> Int16
    scale, offset = 2.75e-05, -0.2
    red = img.select("SR_B4").multiply(scale).add(offset)
    nir = img.select("SR_B5").multiply(scale).add(offset)
    ndvi = nir.subtract(red).divide(nir.add(red)).rename("ndvi")
    return ndvi.multiply(10000).toInt16().rename("ndvi")  # scaled NDVI (Int16)

def summer_ndvi_scaled(state_geom, year):
    start = ee.Date.fromYMD(year, 6, 1)
    end   = ee.Date.fromYMD(year, 9, 1)  # end exclusive (covers Jun–Aug)
    col = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
           .filterDate(start, end)
           .filterBounds(state_geom)
           .map(mask_l8_qapixel)
           .map(add_ndvi_scaled))  # each image becomes a single-band 'ndvi' (Int16)
    return col.median().clip(state_geom)  # still Int16 (median of Int16)

# ------------- States FC and export loop -------------
states = (ee.FeatureCollection("TIGER/2018/States")
          .filter(ee.Filter.inList("STUSPS", NEW_ENGLAND))
          .select(["NAME", "STUSPS"]))

def export_state(state_feat, year):
    name = state_feat.get("NAME").getInfo()
    code = state_feat.get("STUSPS").getInfo()
    geom = state_feat.geometry()

    ndvi = summer_ndvi_scaled(geom, year)  # Int16, scaled by 10000

    desc = f"L8_Summer_NDVI_{SCALE_M}m_{year}_{code}"
    kwargs = {
        "image": ndvi,
        "description": desc,
        "folder": DRIVE_FOLDER,
        "fileNamePrefix": desc,
        "region": geom,
        "scale": SCALE_M,
        "maxPixels": 1_000_000_000,
        "fileFormat": "GeoTIFF",
    }
    if CRS:
        kwargs["crs"] = CRS

    ee.batch.Export.image.toDrive(**kwargs).start()
    print(f"Started export for {name} ({code}) → {desc}")

# Kick off exports
lst = states.toList(states.size())
n   = lst.size().getInfo()
print(f"Exporting {n} New England states for YEAR={YEAR} to Drive folder '{DRIVE_FOLDER}'")
for i in range(n):
    export_state(ee.Feature(lst.get(i)), YEAR)

print("All Earth Engine export tasks started. Monitor in the Tasks tab.")


Exporting 6 New England states for YEAR=2020 to Drive folder 'DATA_EE_DOWNLOAD'
Started export for Rhode Island (RI) → L8_Summer_NDVI_30m_2020_RI
Started export for New Hampshire (NH) → L8_Summer_NDVI_30m_2020_NH
Started export for Vermont (VT) → L8_Summer_NDVI_30m_2020_VT
Started export for Connecticut (CT) → L8_Summer_NDVI_30m_2020_CT
Started export for Maine (ME) → L8_Summer_NDVI_30m_2020_ME
Started export for Massachusetts (MA) → L8_Summer_NDVI_30m_2020_MA
All Earth Engine export tasks started. Monitor in the Tasks tab.
